In [ ]:
# To interact with the Reddit API and fetch comments, posts, and user data.

In [1]:
pip install praw

Note: you may need to restart the kernel to use updated packages.


In [2]:
# To utilize OpenAI's models (like GPT-3) for generating text-based responses.

In [3]:
pip install openai

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import praw
import os
import openai

# Set up Reddit API with PRAW
reddit = praw.Reddit(
    client_id='YOUR_REDDIT_CLIENT_ID',       # Replace this with your actual Client ID
    client_secret='YOUR_REDDIT_CLIENT_SECRET', # Replace this with your actual Client Secret
    user_agent='YOUR_USER_AGENT',             # A description of your bot, e.g., 'Reddit AI bot'
    username='YOUR_REDDIT_USERNAME',          # Your Reddit username
    password='YOUR_REDDIT_PASSWORD'           # Your Reddit password
)

# Fetch the submission (thread) using its ID
submission = reddit.submission(id='Comment_ID')  # Use the post ID from your URL

# Ensure to fetch the comments
submission.comments.replace_more(limit=None)  # Fetch all comments

# Print comment IDs and bodies
for comment in submission.comments.list():
    print(f"Comment ID: {comment.id} | Comment Body: {comment.body}")
# Set up OpenAI API key
openai.api_key = os.getenv('OPENAI_API_KEY')

# Fetch the submission (thread) using its ID
submission = reddit.submission(id='Comment_ID')  # Use the post ID from your URL

# Ensure to fetch the comments
submission.comments.replace_more(limit=None)  # Fetch all comments

# Print comment IDs and bodies
for comment in submission.comments.list():
    print(f"Comment ID: {comment.id} | Comment Body: {comment.body}")

# Function to generate response using OpenAI GPT
def generate_response(comment_text):
    try:
        response = openai.Completion.create(
            engine="text-davinci-003",  # GPT model
            prompt=f"A Reddit user commented: '{comment_text}'. Generate a helpfcul and insightful response to engage with them:",
            max_tokens=100,
            n=1,
            stop=None,
            temperature=0.7
        )
        return response.choices[0].text.strip()
    except Exception as e:
        print(f"Error generating response: {e}")
        return None

# Function to recommend a product based on keywords in comment
def recommend_product(comment_text):
    if "facial recognition" in comment_text.lower():
        return "I recommend trying [Product Name], a powerful face recognition tool! You can check it out here: [Product Link]"
    else:
        return None

# Function to upvote or engage with insightful comments
def boost_engagement(comment):
    if comment.score > 10:  # Arbitrary threshold for upvoting
        comment.upvote()
        print(f"Upvoted comment with {comment.score} karma: {comment.body}")

# Monitor specific subreddit
def monitor_subreddit(subreddit_name):
    subreddit = reddit.subreddit(subreddit_name)
    
    for comment in subreddit.stream.comments(skip_existing=True):  # Streaming new comments
        print(f"New comment: {comment.body}")
        
        # Generate response using GPT
        response = generate_response(comment.body)
        if response:
            comment.reply(response)
            print(f"Replied to comment: {response}")
        
        # Promote a product if relevant
        product_promotion = recommend_product(comment.body)
        if product_promotion:
            comment.reply(product_promotion)
            print(f"Promoted product: {product_promotion}")
        
        # Boost engagement by upvoting high-quality comments
        boost_engagement(comment)

if __name__ == "__main__":
    # Start monitoring a subreddit (e.g., "technology")
    monitor_subreddit("technology")